# Exploración de datos

## Estación en tierra (Uniandes)
 - Lectura de CSV (punto y coma, coma decimal)
 - Alineación a malla uniforme de 10 min (UTC)
 - QC básico de GHI (corte de negativos y techo físico)
 - Suavizado opcional (Savitzky–Golay o media móvil)
 - Cálculo de cielo despejado con pvlib y k_ghi (índice de cielo despejado)
 - Exporta Parquet comprimido y figura de diagnóstico

### Librerías

In [1]:
from __future__ import annotations
import os
from pathlib import Path
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm import tqdm

# gráficos
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# suavizado opcional
from scipy.signal import savgol_filter

# contexto solar
import pvlib

In [8]:
df = pd.read_csv("../data/datos_uniandes/datos_tierra.csv",
                          sep=";",
                          index_col=0,
                          parse_dates=True,
                          decimal=",")

In [9]:
df.head()

,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Humedad relativa del aire 1 [%] - E_AH_REL1,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Presión relativa del aire 1 [hPa] - E_AP_REL1,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Dirección del viento [°] - E_W_D,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Radiación [W/m²] - SRAD,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Temperatura [°C] - T,Sistema FV Andes 80kWp - Lufft WS501 BM: RS485-2 67 - Velocidad del viento [m/s] - E_W_S
Timestamp,,,,,,
2023-09-01 00:00:00,73.896,745.288,163.000,0.000,12.718,1.042
2023-09-01 00:05:00,74.888,745.184,108.532,0.000,12.292,2.858
2023-09-01 00:10:00,76.484,745.140,113.694,0.000,11.932,2.728
2023-09-01 00:15:00,76.674,745.032,110.576,0.144,12.132,2.300
2023-09-01 00:20:00,75.824,744.972,116.006,0.000,12.318,1.346
